In [1]:
from pyvis.network import Network
import networkx as nx
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import re
from collections import Counter

In [13]:
# Carico il dataset
df_fisica = pd.read_csv("../data/articoli_fisica-informatica.csv")

# Carico la lista dei professori dal file txt
with open("../data/professori_dipartimento_inf.txt", "r", encoding="utf-8") as f:
    professori_informatica = [riga.strip() for riga in f]

def normalize_prof(name):
    cognome, nome = [x.strip() for x in name.split(',')]
    return f"{cognome} {nome}"

professori_informatica = set(normalize_prof(n) for n in professori_informatica)
professori_informatica = list(professori_informatica)
professori_informatica.sort()
print(professori_informatica)

FileNotFoundError: [Errno 2] No such file or directory: '../data/articoli_fisica-informatica.csv'

In [9]:
def pulisci_autori(author_string):
    if pd.isna(author_string):
        return []
    
    autori = []
    for a in author_string.split(";"):
        a = a.strip()
        # prendo la parte prima di "("
        a = a.split("(")[0].strip()
        
        # separo Cognome, Nome
        if "," in a:
            cognome, nome = a.split(",", 1)
            autori.append(f"{cognome.strip()} {nome.strip()}")
        else:
            autori.append(a)
    return autori


In [10]:
G = nx.Graph()

for autori_string in df_fisica["Author full names"]:
    lista_autori = pulisci_autori(autori_string)

    # Filtra solo i professori del dipartimento
    autori_prof = [a for a in lista_autori if a in professori_informatica]

    # Aggiungi edge tra tutte le coppie di professori che hanno co-firmato
    for i in range(len(autori_prof)):
        for j in range(i + 1, len(autori_prof)):
            a1, a2 = autori_prof[i], autori_prof[j]

            if G.has_edge(a1, a2):
                G[a1][a2]["weight"] += 1
            else:
                G.add_edge(a1, a2, weight=1)


In [11]:
print("Numero di nodi (professori coinvolti):", G.number_of_nodes())
print("Numero di archi (co-autorship):", G.number_of_edges())

# Mostra i vicini di un professore, se vuoi
for n in G.nodes():
    print(n, "co-autore con:", list(G.neighbors(n)))


Numero di nodi (professori coinvolti): 7
Numero di archi (co-autorship): 5
Serra Giuseppe co-autore con: ['Piazza Carla']
Piazza Carla co-autore con: ['Serra Giuseppe']
Roitero Kevin co-autore con: ['Mizzaro Stefano', 'Della Mea Vincenzo']
Mizzaro Stefano co-autore con: ['Roitero Kevin', 'Della Mea Vincenzo']
Della Mea Vincenzo co-autore con: ['Roitero Kevin', 'Mizzaro Stefano']
Fontana Federico co-autore con: ['Bozzo Enrico']
Bozzo Enrico co-autore con: ['Fontana Federico']


In [7]:

# Crea un grafo pyvis
net = Network(height="750px", width="100%", bgcolor="#ffffff", font_color="black")

# Forza fisica per un layout più bello
net.force_atlas_2based()

# Aggiungi nodi e archi da NetworkX a PyVis
for node in G.nodes():
    net.add_node(
        node,
        label=node,
        title=node,
        color="#4A90E2",   # colore nodi
        shape="dot",
        size=15
    )

for u, v, data in G.edges(data=True):
    weight = data.get("weight", 1)
    net.add_edge(
        u, 
        v, 
        value=weight,
        title=f"Co-authored papers: {weight}"
    )

# Salva in HTML

output_path = "./html/grafo_coauthorship_fisica-informatica.html"
net.write_html(output_path)
print("File generato:", output_path)


File generato: ./html/grafo_coauthorship_fisica-informatica.html
